In [1]:
from transformers import BertJapaneseTokenizer, BertModel

from sentence_transformers import SentenceTransformer
from sentence_transformers import models

import torch
import numpy as np


In [2]:
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
model = BertModel.from_pretrained(MODEL_NAME)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def sentence_to_vector(model, tokenizer, sentence):

  # 文を単語に区切って数字にラベル化
  tokens = tokenizer(sentence) < meta charset = "utf-8" >［"input_ids"]

      # BERTモデルの処理のためtensor型に変換
      input = torch.tensor(tokens).reshape(1, -1)

      # BERTモデルに入力し文のベクトルを取得
      with torch.no_grad():
      outputs= model(input, output_hidden_states=True)
      last_hidden_state = outputs.last_hidden_state < meta charset = "utf-8" >［0]
      averaged_hidden_state = last_hidden_state.sum(dim=0) / len(last_hidden_state)

      return averaged_hidden_state


In [3]:
def sentence_to_vector(model, tokenizer, sentence):
    tokens = tokenizer(sentence, add_special_tokens=True)["input_ids"]
    input = torch.tensor(tokens).unsqueeze(0)
    with torch.no_grad():
        outputs = model(input)
        last_hidden_state = outputs[0]
        averaged_hidden_state = last_hidden_state.mean(dim=0)
    return averaged_hidden_state

In [4]:
def sentence_to_vector(model, tokenizer, sentence):
    tokens = tokenizer(sentence, add_special_tokens=True)["input_ids"]
    input = torch.tensor(tokens).unsqueeze(0)
    with torch.no_grad():
        outputs = model(input)
        last_hidden_state = outputs[0]
        averaged_hidden_state = last_hidden_state.mean(dim=0).unsqueeze(0)
    return averaged_hidden_state


In [5]:
sentence = "我輩は猫である。"
sentence_vector = sentence_to_vector(model, tokenizer, sentence)

In [6]:
def calc_similarity(sentence1, sentence2):
  print("{}\n{}".format(sentence1, sentence2))

  sentence_vector1 = sentence_to_vector(model, tokenizer, sentence1)
  sentence_vector2 = sentence_to_vector(model, tokenizer, sentence2)

  score = torch.nn.functional.cosine_similarity(
      sentence_vector1, sentence_vector2, dim=0).detach().numpy().copy()
  print("類似度：", score)


In [7]:
def calc_similarity(sentence1, sentence2):
    print("{}\n{}".format(sentence1, sentence2))

    sentence_vector1 = sentence_to_vector(model, tokenizer, sentence1)
    sentence_vector2 = sentence_to_vector(model, tokenizer, sentence2)

    # Pad the sentence vectors with zeros to make them the same length
    max_len = max(sentence_vector1.size(0), sentence_vector2.size(0))
    sentence_vector1 = torch.nn.functional.pad(
        sentence_vector1, (0, max_len - sentence_vector1.size(0)))
    sentence_vector2 = torch.nn.functional.pad(
        sentence_vector2, (0, max_len - sentence_vector2.size(0)))

    # Calculate the cosine similarity
    score = torch.nn.functional.cosine_similarity(
        sentence_vector1, sentence_vector2, dim=0).detach().numpy().copy()
    print("Similarity:", score)


In [8]:
sentence1 = "吾輩は猫である"

sentence2 = "私は猫です"

calc_similarity(sentence1, sentence2)


吾輩は猫である
私は猫です


RuntimeError: The size of tensor a (8) must match the size of tensor b (6) at non-singleton dimension 1

In [ ]:
sentence3 = "お昼ご飯にカレーが食べたい"

calc_similarity(sentence1, sentence3)